In [ ]:
def sift_knn(prev_scr,scr):
    img1 = scr        
    img2 = prev_scr 
    sift = cv2.xfeatures2d.SURF_create()
    kp1 = sift.detect(img1,None)
    kp1, des1 = sift.compute(img1, kp1)
    kp2 = sift.detect(img2,None)
    kp2, des2 = sift.compute(img2, kp2)
#     bf = cv2.BFMatcher()
#     matches = bf.knnMatch(des1,des2, k=2)
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    # Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]
    # ratio test as per Lowe's paper
    j=0
    k=0    
    for i, (m1,m2) in enumerate(matches):
        k+=1
        if m1.distance < 0.9 * m2.distance:
            j+=1
            matchesMask[i] = [1,0]
            pt1 = kp1[m1.queryIdx].pt
            pt2 = kp2[m1.trainIdx].pt
#             dist = dist(pt1,pt2)
#             if i % 1 ==0:
                # Draw pairs in purple, to make sure the result is ok
            cv2.circle(img1, (int(pt1[0]),int(pt1[1])), 1, (255,0,255), -1)
            cv2.circle(img1, (int(pt2[0]),int(pt2[1])), 1, (0,255,255), -1)

#     print(j)
#     print(k)
    # Draw match in blue, error in red
    draw_params = dict(matchColor = (255, 0,0),
                       singlePointColor = (0,0,255),
                       matchesMask = matchesMask,
                       flags = 0)

    res = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
    cv2.imshow("res",res)


In [ ]:
import numpy as np
import time
import cv2
from pathInertiaModel import pathInertiaModel
from socket import socket, gethostbyname, AF_INET, SOCK_DGRAM
from sklearn.neighbors import KNeighborsClassifier
import copy
from random import randint
import skimage.measure
import keras
import math
kernel = np.ones((5,5),np.uint8)
import threading

classifier1 = KNeighborsClassifier(n_neighbors=1)
classifier2 = KNeighborsClassifier(n_neighbors=1)
classifier_main = KNeighborsClassifier(n_neighbors=1)
obstPred1= [] 
prev_info_countours1 = []
obstPred2 = [] 
prev_info_countours2 = []
beg1=1
beg2=1
classifier3 = KNeighborsClassifier(n_neighbors=1)
classifier4 = KNeighborsClassifier(n_neighbors=1)
obstPred3= [] 
prev_info_countours3 = []
obstPred4 = [] 
prev_info_countours4 = []
beg3=1
beg4=1
pr1 = None
Pr2 = None
def obstacleNcounters():



           
    PORT_NUMBER = 4000
    SIZE = 1024*1024
#    hostName = gethostbyname( '0.0.0.0' )
#    mySocket = socket( AF_INET, SOCK_DGRAM )
#    mySocket.bind( (hostName, PORT_NUMBER) )
    print ("Test server listening on port {0}\n".format(PORT_NUMBER))
    print("start")
    prev_info_countours = []
    info_countours = list(np.zeros((261680,5,2)))
    vrec = cv2.VideoCapture('v5.mp4',0)
    sendInertia = False
    k = 0#300*6*8
    obstPred = []
    loaded_model = loadModel()
    
    while k:
        ret, scr = vrec.read()
        k-=1
    print("running")
    while True:

        label = []
        out = []
        obstPred = []
 
        start_time = time.time()
        ret, scr = vrec.read()
        if not ret:
            break
    
        scr = cv2.resize(scr, (680,480)) 
        scr = np.array(scr[220:480,:])
        scr = cv2.resize(scr, (680*2,260*2))#1360,520
        blank = cv2.imread('blank.png', 0)
        blank = cv2.resize(blank,(680*2,260*2))
    
        frame = cv2.GaussianBlur(scr, (15, 15), 0)
        scr = cv2.addWeighted(scr,3,frame,-2,0)
        gray = cv2.cvtColor(scr,cv2.COLOR_BGR2GRAY)
        cv2.imshow('scred',scr)


        canny = cv2.Canny(gray,50,100)
        canny = blank-canny
        im2, contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        blank = cv2.drawContours(blank, contours, -1, 0, 1)
        dist_transform = cv2.distanceTransform(canny,cv2.DIST_L2,5)
        ret, blank = cv2.threshold(dist_transform,0.02*dist_transform.max(),255,0)
        scr = cv2.bilateralFilter(scr,20,40,40)
        
#     blank = cv2.dilate(blank,kernel,iterations = 2)
        blank = cv2.morphologyEx(blank, cv2.MORPH_OPEN, kernel,5)
        cv2.imshow("blankobj",blank)
        blank = np.uint8(blank)
        im3, contours1, hierarchy = cv2.findContours(blank, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        scr = cv2.drawContours(scr, contours1, -1,[0,255,255], 3)
        cv2.imshow('print',scr)

        print("FPS: ", 1.0 / (time.time() - start_time))
        q=cv2.waitKey(1)
        if q == 27:
            cv2.destroyAllWindows()
            break
    cv2.destroyAllWindows()
obstacleNcounters()